In [ ]:
safety = True

#whether to save jsons
jsons = False

#delete previous jsons
import os
import glob

def delete_jsons():
    json_files = glob.glob("jsons/*.json")
    if json_files:
        for file in json_files:
            os.remove(file)
        print(f"✓ Deleted {len(json_files)} JSON file(s)")
    else:
        print("✓ No JSON files to delete")

# Delete jsons when jsons = False
if not jsons:
    delete_jsons()

import os
from dotenv import load_dotenv
import requests
import json
from datetime import datetime

print("✓ Required libraries imported successfully!")

load_dotenv()
USER_TOKEN = os.getenv("USER_TOKEN")
headers = {"Authorization": f"Bearer {USER_TOKEN}"}




In [ ]:
#The three getters should return lists of 
def getChannels(headers):
    """
    Retrieve all public channels from the Slack workspace.
    
    Args:
        headers: Authorization headers with Slack token
    
    Returns:
        list: List of channel objects with full metadata (id, name, etc.)
    """
    url = "https://slack.com/api/conversations.list"
    params = {
        "types": "public_channel",
        "limit": 1000
    }
    
    all_channels = []
    cursor = None
    
    # Handle pagination
    while True:
        if cursor:
            params["cursor"] = cursor
        
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        
        if not data.get("ok"):
            return []
        
        all_channels.extend(data.get("channels", []))
        
        cursor = data.get("response_metadata", {}).get("next_cursor")
        if not cursor:
            break
    
    return all_channels


def getPrivateChannels(headers):
    """
    Get all private channels the user is a member of.
    
    Args:
        headers: Authorization headers with Slack token
    
    Returns:
        list: List of private channel objects with full metadata (id, name, etc.)
    """
    url = "https://slack.com/api/conversations.list"
    params = {
        "types": "private_channel",
        "limit": 1000
    }
    
    all_private = []
    cursor = None
    
    # Handle pagination
    while True:
        if cursor:
            params["cursor"] = cursor
        
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        
        if not data.get("ok"):
            return []
        
        all_private.extend(data.get("channels", []))
        
        cursor = data.get("response_metadata", {}).get("next_cursor")
        if not cursor:
            break
    
    return all_private


def getDirectMessages(headers):
    """
    Retrieve ALL Direct Messages (DMs) for the authenticated user.
    Gets all DM conversations and all messages from each conversation.
    
    Args:
        headers: Authorization headers with Slack token
    
    Returns:
        list: List of all DM message objects from all conversations
    """
    # First, get all DM conversations
    url = "https://slack.com/api/conversations.list"
    params = {
        "types": "im",  # 'im' = instant message (DM)
        "limit": 1000
    }
    
    all_dms = []
    cursor = None
    
    # Get all DM conversations with pagination
    while True:
        if cursor:
            params["cursor"] = cursor
        
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        
        if not data.get("ok"):
            return []
        
        all_dms.extend(data.get("channels", []))
        
        cursor = data.get("response_metadata", {}).get("next_cursor")
        if not cursor:
            break
    
    # Now get all messages from each DM conversation
    all_messages = []
    
    for dm in all_dms:
        dm_id = dm.get('id')
        
        # Get all messages from this DM
        history_url = "https://slack.com/api/conversations.history"
        history_params = {
            "channel": dm_id,
            "limit": 1000
        }
        
        history_cursor = None
        
        while True:
            if history_cursor:
                history_params["cursor"] = history_cursor
            
            response = requests.get(history_url, headers=headers, params=history_params)
            data = response.json()
            
            if not data.get("ok"):
                break
            
            messages = data.get("messages", [])
            all_messages.extend(messages)
            
            history_cursor = data.get("response_metadata", {}).get("next_cursor")
            if not history_cursor:
                break
    
    return all_messages


def all_messages_in_channel_to_list(channel): #recieves channel object, returns list of {message_link, text, user, channel}
    """
    Convert all messages from a channel to a structured list format.
    
    Args:
        channel: Channel object containing channel metadata
    
    Returns:
        list: List of formatted message dictionaries
    """
    # TODO: Implement this function
    pass




In [ ]:
# Returns a list of {channel_name, channel_id, text, user, message_link}
def all_messages_from_channels_to_list(channels): 
    """
    Convert all messages from a list of channels to a structured list format.
    
    Args:
        channels: List of channel objects OR list of message objects (for DMs)
    
    Returns:
        list: List of formatted message dictionaries with channel info and permalink
    """
    all_formatted_messages = []
    
    # Check if this is a list of message objects (DMs) or channel objects
    if channels and 'text' in channels[0]:
        # This is already a list of messages (from getDirectMessages)
        for msg in channels:
            formatted_msg = {
                "channel_name": "DM",
                "channel_id": msg.get('channel', 'unknown'),
                "text": msg.get('text', ''),
                "user": msg.get('user', 'unknown'),
                "message_link": None  # DM permalinks would need channel context
            }
            all_formatted_messages.append(formatted_msg)
    else:
        # This is a list of channel objects
        for channel in channels:
            channel_id = channel.get('id')
            channel_name = channel.get('name', 'unknown')
            
            # Get all messages from this channel
            history_url = "https://slack.com/api/conversations.history"
            history_params = {
                "channel": channel_id,
                "limit": 1000
            }
            
            cursor = None
            
            while True:
                if cursor:
                    history_params["cursor"] = cursor
                
                response = requests.get(history_url, headers=headers, params=history_params)
                data = response.json()
                
                if not data.get("ok"):
                    break
                
                messages = data.get("messages", [])
                
                # Format each message
                for msg in messages:
                    msg_ts = msg.get('ts')
                    
                    # Get permalink
                    permalink_url = "https://slack.com/api/chat.getPermalink"
                    permalink_params = {
                        "channel": channel_id,
                        "message_ts": msg_ts
                    }
                    
                    permalink_response = requests.get(permalink_url, headers=headers, params=permalink_params)
                    permalink_data = permalink_response.json()
                    
                    message_link = permalink_data.get("permalink") if permalink_data.get("ok") else None
                    
                    formatted_msg = {
                        "channel_name": channel_name,
                        "channel_id": channel_id,
                        "text": msg.get('text', ''),
                        "user": msg.get('user', 'unknown'),
                        "message_link": message_link
                    }
                    all_formatted_messages.append(formatted_msg)
                
                cursor = data.get("response_metadata", {}).get("next_cursor")
                if not cursor:
                    break
    
    return all_formatted_messages


In [ ]:
# Test API connection first
test_url = "https://slack.com/api/auth.test"
test_response = requests.get(test_url, headers=headers)
test_data = test_response.json()

print("API Authentication Test:")
print(f"  OK: {test_data.get('ok')}")
if test_data.get('ok'):
    print(f"  User: {test_data.get('user')}")
    print(f"  Team: {test_data.get('team')}")
else:
    print(f"  Error: {test_data.get('error')}")
    print(f"  Full response: {test_data}")

print("\nFetching data...")
channels = getChannels(headers) 
print(f"Channels retrieved: {len(channels)}")

private_channels = getPrivateChannels(headers)
print(f"Private channels retrieved: {len(private_channels)}")

dm_messages = getDirectMessages(headers)
print(f"DM messages retrieved: {len(dm_messages)}")

cluster_data = []
cluster_data.extend(all_messages_from_channels_to_list(channels))
cluster_data.extend(all_messages_from_channels_to_list(private_channels))
cluster_data.extend(all_messages_from_channels_to_list(dm_messages))

print(f"\nTotal messages in cluster_data: {len(cluster_data)}")

# Save to JSON file
output_data = {
    "source": "slack_all.ipynb",
    "cluster_data": cluster_data
}

output_file = "./output.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2)

print(f"✓ Data saved to {output_file}")